<a href="https://colab.research.google.com/github/Jijnash22/multimodal-parkinsons-ai/blob/main/notebooks/02_facial_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe==0.10.21


  Using cached mediapipe-0.10.21-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached jaxlib-0.8.2-cp312-cp312-manylinux_2_27_x86_64.whl.metadata (1.3 kB)
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
  Using cached jax-0.8.2-py3-none-any.whl.metadata (13 kB)
  Using cached jax-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.8.1-cp312-cp312-manylinux_2_27_x86_64.whl.metadata (1.3 kB)
  Using cached jax-0.8.0-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.8.0-cp312-cp312-manylinux_2_27_x86_64.whl.metadata (1.3 kB)
  Using cached jax-0.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.7.1-cp312-cp312-manylinux_2_27_x86_64.whl.metadata (1.3 kB)
Using cached mediapipe-0.10.21-cp312-cp312-manylinux_2_28_x86_64.whl (35.6 MB)
Using cached jax-0.7.1-py3-none-any.whl (2.8 MB)
Using cached jaxlib-0.7.1-cp312-cp312-manylinux_2_27_x86_64.whl (81.2 M

In [ ]:
import mediapipe as mp
import cv2
import numpy as np

print("MediaPipe version:", mp.__version__)


MediaPipe version: 0.10.21


In [ ]:
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print("Face Mesh initialized successfully")


Face Mesh initialized successfully


In [ ]:
def eye_aspect_ratio(landmarks, left=True):
    if left:
        idx = [33, 160, 158, 133, 153, 144]
    else:
        idx = [362, 385, 387, 263, 373, 380]

    p = [landmarks[i] for i in idx]

    vertical = (
        np.linalg.norm(np.array(p[1]) - np.array(p[5])) +
        np.linalg.norm(np.array(p[2]) - np.array(p[4]))
    )
    horizontal = np.linalg.norm(np.array(p[0]) - np.array(p[3]))

    return vertical / (2.0 * horizontal + 1e-6)


In [ ]:
def extract_facial_features(video_path):
    cap = cv2.VideoCapture(video_path)

    eye_ratios = []
    facial_motion = []
    mouth_motion = []
    eyebrow_motion = []

    prev_landmarks = None
    blink_count = 0
    blink_threshold = 0.20
    closed = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb)

        if results.multi_face_landmarks:
            lm = results.multi_face_landmarks[0]
            landmarks = [(p.x, p.y) for p in lm.landmark]

            left_ear = eye_aspect_ratio(landmarks, left=True)
            right_ear = eye_aspect_ratio(landmarks, left=False)
            ear = (left_ear + right_ear) / 2
            eye_ratios.append(ear)

            if ear < blink_threshold and not closed:
                blink_count += 1
                closed = True
            elif ear >= blink_threshold:
                closed = False

            if prev_landmarks is not None:
                facial_motion.append(
                    np.linalg.norm(
                        np.array(landmarks) - np.array(prev_landmarks)
                    )
                )

                mouth_motion.append(
                    np.linalg.norm(
                        np.array(landmarks[13]) - np.array(prev_landmarks[13])
                    )
                )

                eyebrow_motion.append(
                    np.linalg.norm(
                        np.array(landmarks[65]) - np.array(prev_landmarks[65])
                    )
                )

            prev_landmarks = landmarks

    cap.release()

    features = np.array([
        blink_count,
        np.mean(eye_ratios) if eye_ratios else 0,
        np.var(eye_ratios) if eye_ratios else 0,
        np.mean(facial_motion) if facial_motion else 0,
        np.var(mouth_motion) if mouth_motion else 0,
        np.var(eyebrow_motion) if eyebrow_motion else 0
    ])

    return features


In [ ]:
test_video = "/content/test_face.mp4"  # change if needed

features = extract_facial_features(test_video)
print("Facial feature vector:", features)
print("Feature dimension:", features.shape)


Facial feature vector: [0 0 0 0 0 0]
Feature dimension: (6,)


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving my_face_video.mp4 to my_face_video.mp4


In [ ]:
import os
os.listdir("/content")


['.config',
 'my_face_video.mp4',
 'M Jijnash Kumar - career Academy.mp4',
 'sample_data']

In [ ]:
test_video = "/content/my_face_video.mp4"  # change if needed

features = extract_facial_features(test_video)
print("Facial feature vector:", features)
print("Feature dimension:", features.shape)

Facial feature vector: [1.00000000e+00 1.69503495e-01 3.38286051e-04 2.79320472e-02
 2.10643963e-06 5.25843469e-07]
Feature dimension: (6,)


In [ ]:
features = extract_facial_features(test_video)

print("Facial feature vector:", features)
print("Feature dimension:", features.shape)


Facial feature vector: [1.00000000e+00 1.69544083e-01 3.37221717e-04 2.78162322e-02
 2.05044581e-06 5.21734404e-07]
Feature dimension: (6,)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

FEATURES_DIR = "/content/drive/MyDrive/IIIT_Nagpur_PD_Project/features"
os.makedirs(FEATURES_DIR, exist_ok=True)

print("Features directory ready:", os.path.exists(FEATURES_DIR))


Features directory ready: True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

BASE_PATH = "/content/drive/MyDrive/IIIT_Nagpur_PD_Project"

print("Project folders:", os.listdir(BASE_PATH))
print("Features:", os.listdir(BASE_PATH + "/features"))


Project folders: ['data', 'notebooks', 'features', 'models']
Features: ['X_features.npy', 'y_labels.npy', 'X_normalized.npy', 'standard_scaler.pkl', 'X_optimized.npy', 'feature_weights_pso.npy', 'voice_embeddings.npy', 'facial_features_demo.npy']


In [ ]:
import numpy as np

BASE_PATH = "/content/drive/MyDrive/IIIT_Nagpur_PD_Project"

facial_features = np.load(
    BASE_PATH + "/features/facial_features_demo.npy"
)

print("Facial features shape:", facial_features.shape)
print("Facial features:", facial_features)


Facial features shape: (6,)
Facial features: [1.00000000e+00 1.69544083e-01 3.37221717e-04 2.78162322e-02
 2.05044581e-06 5.21734404e-07]


In [ ]:
# Create pseudo dataset for encoder training
X_face = np.tile(facial_features, (40, 1))

# Fake labels (20 HC, 20 PD) just to build encoder
y_face = np.array([0]*20 + [1]*20)

print("X_face shape:", X_face.shape)
print("y_face shape:", y_face.shape)


X_face shape: (40, 6)
y_face shape: (40,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

face_model = Sequential([
    Input(shape=(6,)),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),   # 👈 this becomes embedding
    Dense(1, activation='sigmoid')
])

face_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

face_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 32)             │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769 (3.00 KB)

 Trainable params: 769 (3.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
face_model.fit(
    X_face,
    y_face,
    epochs=100,
    batch_size=8,
    verbose=1
)


Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5226 - loss: 0.6920
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4740 - loss: 0.7133 
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5191 - loss: 0.6957 
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5451 - loss: 0.7014 
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4219 - loss: 0.7154 
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4878 - loss: 0.7054 
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4760 - loss: 0.7009 
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5052 - loss: 0.6904 
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5830 - loss: 0.6836 
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5049 - loss: 0.6927 
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4642 - loss: 0.6964 
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4028 - loss

In [ ]:
# Force the model to build by passing a dummy batch
face_model.predict(X_face[:1])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


array([[0.49416867]], dtype=float32)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

face_input = Input(shape=(6,), name="face_input")

x = Dense(32, activation="relu")(face_input)
x = Dropout(0.2)(x)
embedding = Dense(16, activation="relu", name="face_embedding")(x)

output = Dense(1, activation="sigmoid")(embedding)

face_model = Model(inputs=face_input, outputs=output)

face_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

face_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ face_input (InputLayer)         │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ face_embedding (Dense)          │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769 (3.00 KB)

 Trainable params: 769 (3.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
face_model.fit(
    X_face,
    y_face,
    epochs=100,
    batch_size=8,
    verbose=1
)


Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6417 - loss: 0.6831
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4153 - loss: 0.7116
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4653 - loss: 0.6997
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5156 - loss: 0.6876
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6229 - loss: 0.6884
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4451 - loss: 0.6935
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4972 - loss: 0.6881
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5278 - loss: 0.6902
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4535 - loss: 0.6952
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4205 - loss: 0.6949
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5611 - loss: 0.6954 
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6115 - los

In [ ]:
face_embedding_model = Model(
    inputs=face_model.input,
    outputs=face_model.get_layer("face_embedding").output
)

face_embedding_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ face_input (InputLayer)         │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ face_embedding (Dense)          │ (None, 16)             │           528 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 752 (2.94 KB)

 Trainable params: 752 (2.94 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
facial_embeddings = face_embedding_model.predict(X_face)

print("Facial embeddings shape:", facial_embeddings.shape)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Facial embeddings shape: (40, 16)


In [ ]:
import numpy as np

np.save(
    BASE_PATH + "/features/facial_embeddings.npy",
    facial_embeddings
)

face_model.save(
    BASE_PATH + "/models/face_model.keras"
)

face_embedding_model.save(
    BASE_PATH + "/models/face_embedding_model.keras"
)

print("Facial model + embeddings saved")


Facial model + embeddings saved


In [ ]:
!find "/content/drive/MyDrive/IIIT_Nagpur_PD_Project" -type f | grep facial


find: ‘/content/drive/MyDrive/IIIT_Nagpur_PD_Project’: No such file or directory


In [ ]:
!ls /content/drive


ls: cannot access '/content/drive': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive"


'(074), (199)_ M. Jijnash , K. Avinash.pdf'
'(074), (199), M. Jijnash kumar, K. Avinash.pdf'
'(074), (199), _M. Jijnash Kumar, _K. Avinash.pdf'
'2.Nature-based Solutions for Improving Human Well-being.docx'
 3.jpg
'Abstract 2026.pdf'
 APP.xlsx
'A State-of-the-Art Review: Machine Learning Paradigms for Personalized Drug Recommendation and Therapeutic Optimization (1).gdoc'
'A State-of-the-Art Review: Machine Learning Paradigms for Personalized Drug Recommendation and Therapeutic Optimization (2).gdoc'
'A State-of-the-Art Review: Machine Learning Paradigms for Personalized Drug Recommendation and Therapeutic Optimization (3).gdoc'
'A State-of-the-Art Review: Machine Learning Paradigms for Personalized Drug Recommendation and Therapeutic Optimization (4).gdoc'
'A State-of-the-Art Review: Machine Learning Paradigms for Personalized Drug Recommendation and Therapeutic Optimization (5).gdoc'
'A State-of-the-Art Review: Machine Learning Paradigms for Personalized Drug Recommendation and Thera

In [ ]:
!find "/content/drive/MyDrive/IIIT_Nagpur_PD_Project" -type f | grep facial


/content/drive/MyDrive/IIIT_Nagpur_PD_Project/notebooks/02_facial_pipeline.ipynb
/content/drive/MyDrive/IIIT_Nagpur_PD_Project/features/facial_features_demo.npy
/content/drive/MyDrive/IIIT_Nagpur_PD_Project/features/facial_embeddings.npy


In [ ]:
import numpy as np

X_face = np.load("/content/drive/MyDrive/IIIT_Nagpur_PD_Project/features/facial_embeddings.npy")
X_face.shape


(40, 16)

In [ ]:
y_face = np.array([0]*20 + [1]*20)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X_train, X_test, y_train, y_test = train_test_split(X_face, y_face, test_size=0.2, random_state=42)

face_clf = Sequential([
    Dense(16, activation='relu', input_shape=(16,)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

face_clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
face_clf.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5312 - loss: 0.6923 - val_accuracy: 0.3750 - val_loss: 0.7228
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.5312 - loss: 0.6919 - val_accuracy: 0.3750 - val_loss: 0.7195
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.5312 - loss: 0.6916 - val_accuracy: 0.3750 - val_loss: 0.7167
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.5312 - loss: 0.6914 - val_accuracy: 0.3750 - val_loss: 0.7141
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.5312 - loss: 0.6912 - val_accuracy: 0.3750 - val_loss: 0.7117
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.5312 - loss: 0.6912 - val_accuracy: 0.3750 - val_loss: 0.7096
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.5312 - loss: 0.6912 - val_accuracy: 0.3750 - val_loss: 0.7079
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.5312 - loss: 0.6912 - val_accuracy: 0.3750 - val_loss: 0.7067
Epoch 9/50
1

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_prob = face_clf.predict(X_test).ravel()
y_pred = (y_prob > 0.5).astype(int)

print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1       :", f1_score(y_test, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
Accuracy : 0.375
Precision: 0.375
Recall   : 1.0
F1       : 0.5454545454545454


In [ ]:
face_clf.save("/content/drive/MyDrive/IIIT_Nagpur_PD_Project/models/face_pd_classifier.keras")
print("Face PD classifier saved")


Face PD classifier saved
